In [4]:
import tkinter as tk
from tkinter import ttk
import time

class SudokuSolver:
    def __init__(self, puzzle):
        self.puzzle = puzzle

    def solve_with_backtracking(self):
        solver = SudokuCSP(self.puzzle)
        if solver.solve_sudoku():
            self.puzzle = solver.puzzle
            return True
        else:
            return False

    def solve_with_ac3(self):
        solver = SudokuCSP_AC3(self.puzzle)
        if solver.solve_sudoku():
            self.puzzle = solver.puzzle
            return True
        else:
            return False

    def solve_puzzle(self, algorithm):
        if algorithm == "Backtracking":
            return self.solve_with_backtracking()
        elif algorithm == "AC-3":
            return self.solve_with_backtracking()

class SudokuGUI:
    def __init__(self):
        self.root = tk.Tk()
        self.root.title("Sudoku Solver")

        self.algorithm_var = tk.StringVar()
        self.difficulty_var = tk.StringVar()
        self.time_taken_var = tk.StringVar()
        self.board_canvas = None
        self.puzzle = None

        self.create_widgets()

    def create_widgets(self):
        # Main frame for controls
        control_frame = ttk.Frame(self.root, padding=(20, 10))
        control_frame.grid(row=0, column=0, columnspan=3, sticky="nsew")

        # Select Algorithm Label and Radio Buttons
        algorithm_label = ttk.Label(control_frame, text="Select Algorithm:", font=("Arial", 12, "bold"))
        algorithm_label.grid(row=0, column=0, padx=10, pady=10, sticky=tk.W)

        # Radio Buttons for Algorithm Selection
        algorithm_options = [("Backtracking", "Backtracking"), ("AC-3", "AC-3")]

        self.algorithm_var.set("Backtracking")  # Set default algorithm
        for i, (algo_text, algo_value) in enumerate(algorithm_options):
            algo_radio = ttk.Radiobutton(control_frame, text=algo_text, value=algo_value, variable=self.algorithm_var)
            algo_radio.grid(row=0, column=i + 1, padx=10, pady=10, sticky=tk.W)

        # Select Difficulty Label and Combobox
        difficulty_label = ttk.Label(control_frame, text="Select Difficulty:", font=("Arial", 12, "bold"))
        difficulty_label.grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)
        difficulty_options = ttk.Combobox(control_frame, textvariable=self.difficulty_var, values=["Easy", "Medium", "Hard"], font=("Arial", 12))
        difficulty_options.grid(row=1, column=1, padx=10, pady=10, sticky=tk.W)
        difficulty_options.current(0)

        # Load Puzzle Button
        load_button = ttk.Button(control_frame, text="Load Puzzle", command=self.load_puzzle)
        load_button.grid(row=1, column=2, padx=10, pady=10, sticky=tk.E)

        # Solve Puzzle Button
        solve_button = ttk.Button(control_frame, text="Solve Puzzle", command=self.solve_and_display)
        solve_button.grid(row=2, column=0, columnspan=3, padx=20, pady=20, sticky=tk.EW)

        # Time Taken Label
        time_taken_label = ttk.Label(self.root, text="Time Taken:", font=("Arial", 12, "bold"))
        time_taken_label.grid(row=1, column=0, padx=10, pady=10, sticky=tk.W)

        # Time Taken Display Label
        time_taken_display = ttk.Label(self.root, textvariable=self.time_taken_var, font=("Arial", 12))
        time_taken_display.grid(row=1, column=1, padx=10, pady=10, sticky=tk.W)

        # Sudoku Board Canvas
        self.board_canvas = tk.Canvas(self.root, bg="white", bd=0, highlightthickness=0)
        self.board_canvas.grid(row=3, column=0, columnspan=3, padx=20, pady=20)

        # Configure row and column weights for resizing
        self.root.rowconfigure(3, weight=1)
        self.root.columnconfigure(0, weight=1)

    def load_puzzle(self):
        difficulty = self.difficulty_var.get()

        # Define puzzles for different difficulties
        puzzles = {
            "Easy": [
                [1, 2, 0, 0, 0, 8, 0, 3, 0],
                [7, 0, 0, 9, 0, 0, 8, 0, 0],
                [0, 0, 8, 3, 0, 5, 0, 0, 0],
                [0, 5, 0, 0, 7, 0, 0, 0, 6],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [6, 0, 0, 0, 9, 0, 0, 7, 0],
                [0, 0, 0, 1, 0, 4, 5, 0, 0],
                [0, 0, 3, 0, 0, 9, 0, 0, 8],
                [0, 8, 0, 6, 0, 0, 0, 4, 0]
            ],
            "Medium": [
                [5, 3, 0, 0, 7, 0, 0, 0, 0],
                [6, 0, 0, 1, 9, 5, 0, 0, 0],
                [0, 9, 8, 0, 0, 0, 0, 6, 0],
                [8, 0, 0, 0, 6, 0, 0, 0, 3],
                [4, 0, 0, 8, 0, 3, 0, 0, 1],
                [7, 0, 0, 0, 2, 0, 0, 0, 6],
                [0, 6, 0, 0, 0, 0, 2, 8, 0],
                [0, 0, 0, 4, 1, 9, 0, 0, 5],
                [0, 0, 0, 0, 8, 0, 0, 7, 9]
            ],
            "Hard": [
                [3, 6, 0, 0, 0, 8, 0, 5, 0],
                [7, 0, 0, 9, 0, 0, 8, 0, 0],
                [0, 0, 8, 3, 0, 5, 0, 0, 0],
                [0, 5, 0, 0, 7, 0, 0, 0, 6],
                [0, 0, 0, 0, 0, 0, 0, 0, 0],
                [6, 0, 0, 0, 9, 0, 0, 7, 0],
                [0, 0, 0, 1, 0, 4, 5, 0, 0],
                [0, 0, 3, 0, 0, 9, 0, 0, 8],
                [0, 8, 0, 6, 0, 0, 0, 4, 0]
            ]
        }

        # Load the puzzle based on difficulty selected
        if difficulty in puzzles:
            self.puzzle = puzzles[difficulty]
            self.display_board(self.puzzle)
        else:
            # Default to an easy puzzle if difficulty is unrecognized
            self.puzzle = puzzles["Easy"]
            self.display_board(self.puzzle)

    def solve_and_display(self):
        if not self.puzzle:
            return

        solver = SudokuSolver(self.puzzle)

        start_time = time.time()
        success = solver.solve_puzzle(self.algorithm_var.get())
        end_time = time.time()

        time_taken = end_time - start_time
        self.time_taken_var.set(f"{time_taken:.4f} seconds")

        if success:
            self.animate_solution(solver.puzzle)

    def animate_solution(self, solution):
        if not self.board_canvas:
            return

        cell_size = 50
        board_size = cell_size * 9

        # Define a list of colors to alternate
        colors = ['red', 'blue', 'green', 'orange', 'purple']

        for i in range(9):
            for j in range(9):
                x0, y0 = j * cell_size, i * cell_size
                x1, y1 = x0 + cell_size, y0 + cell_size

                # Draw the cell border with alternating colors
                color_index = (i * 9 + j) % len(colors)
                color = colors[color_index]
                self.board_canvas.create_rectangle(x0, y0, x1, y1, outline=color)

                # Display the puzzle number in the cell with animation delay
                self.root.after(1000 * (i * 9 + j), self.draw_cell_value, (x0 + cell_size / 2, y0 + cell_size / 2), str(solution[i][j]), cell_size)

    def draw_cell_value(self, position, value, cell_size):
        x, y = position
        self.board_canvas.create_text(x, y, text=value, font=("Arial", 20))

    def display_board(self, puzzle):
        if self.board_canvas:
            self.board_canvas.destroy()

        cell_size = 50
        board_size = cell_size * 9

        self.board_canvas = tk.Canvas(self.root, width=board_size, height=board_size, bg="white")
        self.board_canvas.grid(row=3, column=0, columnspan=3, padx=20, pady=20)

        for i in range(9):
            for j in range(9):
                x0, y0 = j * cell_size, i * cell_size
                x1, y1 = x0 + cell_size, y0 + cell_size

                # Draw the cell border
                self.board_canvas.create_rectangle(x0, y0, x1, y1, outline="black")

                # Display the puzzle number in the cell
                cell_value = puzzle[i][j]
                if cell_value != 0:
                    self.board_canvas.create_text(x0 + cell_size / 2, y0 + cell_size / 2, text=str(cell_value), font=("Arial", 20))

    def run(self):
        self.root.mainloop()

if __name__ == "__main__":
    app = SudokuGUI()
    app.run()


In [1]:

from collections import deque

class SudokuCSP_AC3:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.assignment = self.initialize_assignment()

    def initialize_assignment(self):
        assignment = {}
        for i in range(9):
            for j in range(9):
                if self.puzzle[i][j] != 0:
                    assignment[(i, j)] = {self.puzzle[i][j]}
                else:
                    assignment[(i, j)] = {1, 2, 3, 4, 5, 6, 7, 8, 9}  # Initial domain
        return assignment

    def is_consistent(self, row, col, num):
        return self.is_valid_row(row, num) and self.is_valid_col(col, num) and self.is_valid_box(row - row % 3, col - col % 3, num)

    def is_valid_row(self, row, num):
        row_index = row[0]  # Extract the row index from the tuple
        return num not in self.puzzle[row_index]

    def is_valid_col(self, col, num):
        return num not in [self.puzzle[i][col] for i in range(9)]

    def is_valid_box(self, startRow, startCol, num):
        for i in range(startRow, startRow + 3):
            for j in range(startCol, startCol + 3):
                if self.puzzle[i][j] == num:
                    return False
        return True

    def solve_sudoku(self):
        if self.ac3():
            return self.assignment
        else:
            return None

    def ac3(self):
        queue = deque(self.get_all_arcs())
        
        while queue:
            xi, xj = queue.popleft()
            if self.revise(xi, xj):
                if len(self.assignment[xi]) == 0:
                    return False
                for xk in self.get_neighbors(xi):
                    if xk != xj:
                        queue.append((xk, xi))
        return True

    def revise(self, xi, xj):
        revised = False
        for xi_val in list(self.assignment[xi]):
            if not any(self.is_consistent(xi, xj, xj_val) for xj_val in self.assignment[xj]):
                self.assignment[xi].remove(xi_val)
                revised = True
        return revised

    def get_all_arcs(self):
        arcs = []
        for i in range(9):
            for j in range(9):
                neighbors = self.get_neighbors((i, j))
                for neighbor in neighbors:
                    arcs.append(((i, j), neighbor))
        return arcs

    def get_neighbors(self, cell):
        row, col = cell
        neighbors = []
        for i in range(9):
            if i != col:
                neighbors.append((row, i))
            if i != row:
                neighbors.append((i, col))
        startRow, startCol = row - row % 3, col - col % 3
        for i in range(startRow, startRow + 3):
            for j in range(startCol, startCol + 3):
                if (i, j) != cell:
                    neighbors.append((i, j))
        return neighbors

In [2]:
from collections import defaultdict

class SudokuCSP:
    def __init__(self, puzzle):
        self.puzzle = puzzle
        self.assignment = {}

    def is_consistent(self, row, col, num):
        return self.is_valid_row(row, num) and self.is_valid_col(col, num) and self.is_valid_box(row - row % 3, col - col % 3, num)

    def is_valid_row(self, row, num):
        return num not in self.puzzle[row]

    def is_valid_col(self, col, num):
       return num not in [self.puzzle[i][col] for i in range(9)]

    def is_valid_box(self, startRow, startCol, num):
        
        for i in range(startRow, startRow + 3):
            for j in range(startCol, startCol + 3):
                if self.puzzle[i][j] == num:
                    return False
        return True

    def solve_sudoku(self):
        return self.func_backtrack()

    def func_backtrack(self):
        # Find the first unassigned cell
        row, col = self.find_unassigned_cell()
        
        if row == -1 and col == -1:
            return True  # All cells are assigned
        
        for num in range(1, 10):  # Numbers from 1 to 9
            if self.is_consistent(row, col, num):
                self.puzzle[row][col] = num  # Assign the number
                
                if self.func_backtrack():
                    return True
                
                # If assigning 'num' doesn't lead to a solution, backtrack
                self.puzzle[row][col] = 0
        
        return False  

    def find_unassigned_cell(self):
        # Find the first unassigned cell in the puzzle
        for i in range(9):
            for j in range(9):
                if self.puzzle[i][j] == 0:
                    return i, j
        return -1, -1  # If all cells are assigned, return (-1, -1)

